# Financial Data Exploratory Visualization
This notebook loads and visualizes various financial datasets using Plotly.

In [ ]:
# Import Required Libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Set data directory
data_dir = './'

# Load all datasets
df_tlt = pd.read_csv(data_dir + 'TLT_2007-01-01_to_2025-08-30.csv', parse_dates=['Date'])
df_ust10y = pd.read_csv(data_dir + 'UST10Y.csv', parse_dates=['Date'])
df_vix = pd.read_csv(data_dir + 'VIX.csv', parse_dates=['Date'])
df_breakeven = pd.read_csv(data_dir + 'Breakeven10Y.csv', parse_dates=['Date'])
df_fedfunds = pd.read_csv(data_dir + 'FedFunds.csv', parse_dates=['Date'])
df_termspread = pd.read_csv(data_dir + 'TermSpread_10Y_2Y.csv', parse_dates=['Date'])

# Load MOVE with proper handling
df_move = pd.read_csv(data_dir + 'MOVE_index.csv', parse_dates=['Date'], skiprows=[1])

df_nfci = pd.read_csv(data_dir + 'NFCI.csv', parse_dates=['Date'])
df_hyoas = pd.read_csv(data_dir + 'HYOAS.csv', parse_dates=['Date'])

print("Datasets loaded successfully!")
print(f"TLT: {len(df_tlt)} rows, Date range: {df_tlt['Date'].min()} to {df_tlt['Date'].max()}")
print(f"UST10Y: {len(df_ust10y)} rows, Date range: {df_ust10y['Date'].min()} to {df_ust10y['Date'].max()}")
print(f"VIX: {len(df_vix)} rows")
print(f"MOVE: {len(df_move)} rows")
print(f"NFCI: {len(df_nfci)} rows")

## 1. TLT (Treasury Bond ETF) Price and Realized Volatility

In [ ]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("TLT Price Over Time", "TLT Realized Volatility (20-day)"),
    specs=[[{"secondary_y": False}], [{"secondary_y": False}]],
    vertical_spacing=0.12
)

# Price subplot
fig.add_trace(
    go.Scatter(x=df_tlt['Date'], y=df_tlt['Price'], 
               name='TLT Price', line=dict(color='steelblue', width=1.5),
               hovertemplate='%{x|%Y-%m-%d}<br>Price: $%{y:.2f}<extra></extra>'),
    row=1, col=1
)

# Realized Vol subplot
vol_clean = df_tlt['Realized_Vol'].dropna()
fig.add_trace(
    go.Scatter(x=vol_clean.index.map(lambda i: df_tlt.loc[i, 'Date']), y=vol_clean,
               name='Realized Vol', line=dict(color='crimson', width=1.5),
               hovertemplate='%{x|%Y-%m-%d}<br>Realized Vol: %{y:.4f}<extra></extra>'),
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Price ($)", row=1, col=1)
fig.update_yaxes(title_text="Volatility", row=2, col=1)
fig.update_layout(height=700, title_text="TLT Performance Analysis", hovermode='x unified')
fig.show()

## 2. Interest Rates: UST 10Y vs Fed Funds

In [ ]:
fig = go.Figure()

# Remove rows with empty values
ust_clean = df_ust10y.dropna(subset=['UST10Y'])
ff_clean = df_fedfunds.dropna(subset=['FedFunds'])

fig.add_trace(go.Scatter(
    x=ust_clean['Date'], y=ust_clean['UST10Y'],
    name='10Y US Treasury Yield',
    line=dict(color='darkblue', width=2),
    hovertemplate='%{x|%Y-%m-%d}<br>UST 10Y: %{y:.2f}%<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=ff_clean['Date'], y=ff_clean['FedFunds'],
    name='Fed Funds Rate',
    line=dict(color='orange', width=2),
    hovertemplate='%{x|%Y-%m-%d}<br>Fed Funds: %{y:.2f}%<extra></extra>'
))

fig.update_layout(
    title='US Interest Rates: 10Y Treasury vs Fed Funds Rate',
    xaxis_title='Date',
    yaxis_title='Rate (%)',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 3. Term Spread (10Y - 2Y) Analysis

In [ ]:
ts_clean = df_termspread.dropna(subset=['TermSpread_10Y_2Y'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ts_clean['Date'], y=ts_clean['TermSpread_10Y_2Y'],
    name='Term Spread',
    fill='tozeroy',
    line=dict(color='darkgreen', width=2),
    hovertemplate='%{x|%Y-%m-%d}<br>Spread: %{y:.2f}%<extra></extra>'
))

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="Zero Line")

fig.update_layout(
    title='Term Spread (10Y - 2Y Treasury)',
    xaxis_title='Date',
    yaxis_title='Spread (%)',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 4. Volatility Indicators: VIX vs MOVE Index

In [ ]:
vix_clean = df_vix.dropna(subset=['VIX'])
move_clean = df_move.dropna(subset=['MOVE_Close'])

# Create secondary axis for MOVE (different scales)
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=vix_clean['Date'], y=vix_clean['VIX'],
               name='VIX (Equity Volatility)',
               line=dict(color='red', width=2),
               hovertemplate='%{x|%Y-%m-%d}<br>VIX: %{y:.2f}<extra></extra>'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=move_clean['Date'], y=move_clean['MOVE_Close'],
               name='MOVE (Bond Volatility)',
               line=dict(color='purple', width=2),
               hovertemplate='%{x|%Y-%m-%d}<br>MOVE: %{y:.2f}<extra></extra>'),
    secondary_y=True,
)

fig.update_yaxes(title_text="VIX", secondary_y=False)
fig.update_yaxes(title_text="MOVE Index", secondary_y=True)
fig.update_xaxes(title_text="Date")

fig.update_layout(
    title='Volatility Indicators: VIX vs MOVE Index',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 5. Inflation Expectations: Breakeven Inflation Rate

In [ ]:
be_clean = df_breakeven.dropna(subset=['Breakeven10Y'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=be_clean['Date'], y=be_clean['Breakeven10Y'],
    name='10Y Breakeven Inflation',
    fill='tozeroy',
    line=dict(color='teal', width=2),
    hovertemplate='%{x|%Y-%m-%d}<br>Breakeven: %{y:.2f}%<extra></extra>'
))

fig.update_layout(
    title='10-Year Breakeven Inflation Rate',
    xaxis_title='Date',
    yaxis_title='Inflation Expectation (%)',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 6. Credit Risk: High Yield OAS

In [ ]:
hyoas_clean = df_hyoas.dropna(subset=['HYOAS'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=hyoas_clean['Date'], y=hyoas_clean['HYOAS'],
    name='HY OAS',
    fill='tozeroy',
    line=dict(color='darkred', width=2),
    hovertemplate='%{x|%Y-%m-%d}<br>HY OAS: %{y:.2f} bps<extra></extra>'
))

fig.update_layout(
    title='High Yield Option-Adjusted Spread (OAS)',
    xaxis_title='Date',
    yaxis_title='OAS (basis points)',
    height=500,
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 7. Financial Conditions Index (NFCI)

In [ ]:
nfci_clean = df_nfci.dropna(subset=['NFCI'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=nfci_clean['Date'], y=nfci_clean['NFCI'],
    name='NFCI',
    fill='tozeroy',
    line=dict(color='navy', width=2),
    hovertemplate='%{x|%Y-%m-%d}<br>NFCI: %{y:.5f}<extra></extra>'
))

fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Zero (Normal Conditions)")

fig.update_layout(
    title='Chicago Fed National Financial Conditions Index (NFCI)',
    xaxis_title='Date',
    yaxis_title='NFCI Score',
    height=500,
    hovermode='x unified',
    template='plotly_white',
    annotations=[dict(text="Positive values indicate tighter financial conditions", xref="paper", yref="paper",
                      x=0.02, y=0.98, showarrow=False, xanchor='left', yanchor='top')]
)
fig.show()

## 8. Comprehensive Multi-Asset Dashboard

In [ ]:
# Normalize data for comparison
ust_norm = (ust_clean['UST10Y'].values - ust_clean['UST10Y'].mean()) / ust_clean['UST10Y'].std()
vix_norm = (vix_clean['VIX'].values - vix_clean['VIX'].mean()) / vix_clean['VIX'].std()
move_norm = (move_clean['MOVE_Close'].values - move_clean['MOVE_Close'].mean()) / move_clean['MOVE_Close'].std()
be_norm = (be_clean['Breakeven10Y'].values - be_clean['Breakeven10Y'].mean()) / be_clean['Breakeven10Y'].std()

fig = go.Figure()

fig.add_trace(go.Scatter(x=ust_clean['Date'], y=ust_norm, name='UST 10Y (normalized)', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=vix_clean['Date'], y=vix_norm, name='VIX (normalized)', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=move_clean['Date'], y=move_norm, name='MOVE (normalized)', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=be_clean['Date'], y=be_norm, name='Breakeven 10Y (normalized)', line=dict(width=1.5)))

fig.add_hline(y=0, line_dash="dash", line_color="gray", annotation_text="Mean")

fig.update_layout(
    title='Normalized Financial Indicators Comparison',
    xaxis_title='Date',
    yaxis_title='Normalized Value (Z-scores)',
    height=600,
    hovermode='x unified',
    template='plotly_white'
)
fig.show()

## 9. Data Quality Summary

In [ ]:
# Create a data quality summary
summary_data = {
    'Dataset': ['TLT Price', 'UST 10Y', 'VIX', 'Breakeven 10Y', 'Fed Funds', 'Term Spread', 'MOVE', 'NFCI', 'HY OAS'],
    'Total Rows': [len(df_tlt), len(df_ust10y), len(df_vix), len(df_breakeven), len(df_fedfunds), 
                   len(df_termspread), len(df_move), len(df_nfci), len(df_hyoas)],
    'Non-Null': [df_tlt['Price'].notna().sum(), df_ust10y['UST10Y'].notna().sum(), df_vix['VIX'].notna().sum(),
                 df_breakeven['Breakeven10Y'].notna().sum(), df_fedfunds['FedFunds'].notna().sum(),
                 df_termspread['TermSpread_10Y_2Y'].notna().sum(), df_move['MOVE_Close'].notna().sum(),
                 df_nfci['NFCI'].notna().sum(), df_hyoas['HYOAS'].notna().sum()],
    'Missing %': [
        (1 - df_tlt['Price'].notna().sum() / len(df_tlt)) * 100,
        (1 - df_ust10y['UST10Y'].notna().sum() / len(df_ust10y)) * 100,
        (1 - df_vix['VIX'].notna().sum() / len(df_vix)) * 100,
        (1 - df_breakeven['Breakeven10Y'].notna().sum() / len(df_breakeven)) * 100,
        (1 - df_fedfunds['FedFunds'].notna().sum() / len(df_fedfunds)) * 100,
        (1 - df_termspread['TermSpread_10Y_2Y'].notna().sum() / len(df_termspread)) * 100,
        (1 - df_move['MOVE_Close'].notna().sum() / len(df_move)) * 100,
        (1 - df_nfci['NFCI'].notna().sum() / len(df_nfci)) * 100,
        (1 - df_hyoas['HYOAS'].notna().sum() / len(df_hyoas)) * 100
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n=== DATA QUALITY SUMMARY ===")
print(summary_df.to_string(index=False))

# Create a visual summary
fig = go.Figure(data=[
    go.Bar(x=summary_df['Dataset'], y=summary_df['Missing %'], name='Missing %', marker_color='lightcoral')
])

fig.update_layout(
    title='Data Completeness by Dataset',
    xaxis_title='Dataset',
    yaxis_title='Missing Data (%)',
    height=400,
    template='plotly_white'
)
fig.show()

## 10. Statistical Summary

In [ ]:
print("\n=== STATISTICAL SUMMARY ===")
print("\nTLT Price:")
print(df_tlt['Price'].describe())

print("\nUST 10Y Yield:")
print(df_ust10y['UST10Y'].describe())

print("\nVIX:")
print(df_vix['VIX'].describe())

print("\nMOVE Index:")
print(df_move['MOVE_Close'].describe())

print("\nTerm Spread:")
print(df_termspread['TermSpread_10Y_2Y'].describe())

print("\nHY OAS:")
print(df_hyoas['HYOAS'].describe())